## Data Lagging

To address the data lagging issue in our analysis of the relationship between CO2 and the number of occupants, we need to consider the delay between people entering or exiting the room and the resulting change in CO2 levels. This delay occurs because it takes a certain amount of time for the CO2 levels to populate the room after individuals have entered or left.

To quantify this lag, we introduce the concept of the `upper bound value (UB)`. The UB represents the maximum CO2 value that can be reached in the room and is calculated based on the room volume. By determining the UB, we can estimate the time lag needed to account for the delay between occupancy changes and CO2 level changes.

The UB is calculated as follows:

$$UB = \left|\frac{RL \times RW \times RH}{C}\right|$$

where:  
- $UB$ : Upper Bound Value
- $RL$ : Room Length 
- $RW$ : Room Width
- $RH$ : Room Height
- $C$ : Constant (100)


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
# def calculate_upper_bound(rl, rw, rh, c=100):
#     ub = abs((rl * rw * rh) / c)
#     return ub

def calculate_upper_bound(rl, rw, rh, c=100):
    # use tensorflow
    rl = tf.constant(rl, dtype=tf.float32)
    rw = tf.constant(rw, dtype=tf.float32)
    rh = tf.constant(rh, dtype=tf.float32)
    c = tf.constant(c, dtype=tf.float32)
    
    ub = tf.abs((rl * rw * rh) / c)
    return ub

To calculate a line of best fit, we need to calculate the slope value between CO2 and occupancy data.  

$$SL = \frac{\sum(O_t - \bar{O_t})(C_t - \bar{C_t})}{\sum(O_t - \bar{O_t})^2}$$

where:
- $SL$ : Slope of the linear regression line
- $O_t$ : Occupancy value
- $\bar{O_t}$ : Sample means of the known occupancy value
- $C_t$ : CO2 value
- $\bar{C_t}$ : Sample means of the known CO2 value

In [3]:
def calculate_slope(occupancy_values, co2_values):

    occupancy_values = tf.constant(occupancy_values, dtype=tf.float32)
    co2_values = tf.constant(co2_values, dtype=tf.float32)
    
    # calculate the mean of occupancy values
    occupancy_mean = tf.reduce_mean(occupancy_values)
    co2_mean = tf.reduce_mean(co2_values)
    
    # calculate the numerator
    numerator = tf.reduce_sum(
        (occupancy_values - occupancy_mean) * (co2_values - co2_mean))
    
    # calculate the denominator
    denominator = tf.reduce_sum(
        (occupancy_values - occupancy_mean) ** 2)
    
    # calculate the slope
    slope = numerator / denominator
    
    return slope

Let's now calculate the intercept value of the linear regression line.

$$IC = \bar{C_t} - SL \times \bar{O_t}$$

where:
- $IC$ : Intercept of the linear regression line
- $\bar{C_t}$ : Sample means of the known CO2 value
- $SL$ : Slope of the linear regression line
- $\bar{O_t}$ : Sample means of the known occupancy value


In [4]:
def calculate_intecept_value(occupancy_values, co2_values, slope):
    
    occupancy_values = tf.constant(occupancy_values, dtype=tf.float32)
    co2_values = tf.constant(co2_values, dtype=tf.float32)
    
    # calculate the mean of occupancy values
    occupancy_mean = tf.reduce_mean(occupancy_values)
    co2_mean = tf.reduce_mean(co2_values)
    
    # calculate the intercept
    intercept = co2_mean - (slope * occupancy_mean)
    
    return intercept

Finally, the formula for the line of best fit is:

$$LBF = (O_t - (SL \times C_t + IC))^2$$

where:
- $LBF$ : Line of best fit
- $O_t$ : Occupancy value
- $SL$ : Slope of the linear regression line
- $C_t$ : CO2 value
- $IC$ : Intercept of the linear regression line


In [5]:
def calculate_line_of_best_fit(occupancy_value, co2_value, slope, intercept):
    
    occupancy_value = tf.constant(occupancy_value, dtype=tf.float32)
    co2_value = tf.constant(co2_value, dtype=tf.float32)
    
    # calculate the line of best fit
    line_of_best_fit = (occupancy_value - (slope * co2_value + intercept)) ** 2
    
    return line_of_best_fit

<!-- write in red to indicate that this place needs to be completed -->
# TODO:
- complete the data lagging section after answers from Irvan

## Seasonal-Trend Decomposition

STD is a method to decompose a time series into three components: trend, seasonal, and residual.  
`The trend component` $\textcolor{orange}{(T_t)}$ is the long-term progression of the series.  
`The seasonal component` $\textcolor{orange}{(S_t)}$ is the repeating short-term cycle in the series.  
`The cyclical component` $\textcolor{orange}{(C_t)}$ is the repeating up and down movements in the series that are not of a fixed period.  
`The residual component` $\textcolor{orange}{(e_t)}$ is the random variation in the series, and is calculated as the difference between the original series and the sum of the trend, seasonal, and cyclical components.

In the paper we're implementing, the cyclical component is combined with the trend component to make a simpler model.

The formula for the STD is:

In [1]:
def calculate_moving_average(df,col):
    moving_average = df[col].rolling(window=12).mean().rolling(window=2).mean()
    return moving_average


def decompose_time_series(df):
    # Step 1: Calculate 2 × 12 moving average in the raw data
    trend_co2 = df['CO2'].rolling(window=12, center=True).mean()
    trend_occupancy = df['occupancy'].rolling(window=12, center=True).mean()

    # Step 2: Calculate centred ratios
    centred_ratios_co2 = df['CO2'] / trend_co2
    centred_ratios_occupancy = df['occupancy'] / trend_occupancy

    for _ in range(2):
        # Step 3: Calculate rough seasonal feature
        seasonal_co2 = centred_ratios_co2.rolling(window=2).mean().rolling(window=2).mean()
        seasonal_occupancy = centred_ratios_occupancy.rolling(window=2).mean().rolling(window=2).mean()

        # Step 4: Calculate irregular feature
        et_co2 = centred_ratios_co2 / seasonal_co2
        et_occupancy = centred_ratios_occupancy / seasonal_occupancy

        # Step 5: Modify irregular feature by multiplying with seasonal feature
        modified_et_co2 = et_co2 * seasonal_co2
        modified_et_occupancy = et_occupancy * seasonal_occupancy

        # Step 6: Update seasonal feature
        seasonal_co2 = modified_et_co2.rolling(window=2).mean().rolling(window=2).mean()
        seasonal_occupancy = modified_et_occupancy.rolling(window=2).mean().rolling(window=2).mean()

        # Step 7: Calculate preliminary seasonal adjusted series
        preliminary_seasonal_adjusted_co2 = df['CO2'] / seasonal_co2
        preliminary_seasonal_adjusted_occupancy = df['occupancy'] / seasonal_occupancy

        # Step 8: Estimate trend feature using weighted Henderson moving average
        trend_co2 = preliminary_seasonal_adjusted_co2.rolling(window=12, center=True).mean()
        trend_occupancy = preliminary_seasonal_adjusted_occupancy.rolling(window=12, center=True).mean()

        # Step 9: Calculate new centred ratios
        centred_ratios_co2 = df['CO2'] / trend_co2
        centred_ratios_occupancy = df['occupancy'] / trend_occupancy

        for _ in range(2):
            # Step 10: Calculate rough seasonal feature using 3 × 5 moving average
            seasonal_co2 = centred_ratios_co2.rolling(window=3, center=True).mean().rolling(window=5, center=True).mean()
            seasonal_occupancy = centred_ratios_occupancy.rolling(window=3, center=True).mean().rolling(window=5, center=True).mean()

            # Step 11: Update irregular feature using modified centred ratios and 3 × 5 moving average
            et_co2 = centred_ratios_co2 / seasonal_co2
            et_occupancy = centred_ratios_occupancy / seasonal_occupancy

            modified_et_co2 = et_co2 * seasonal_co2
            modified_et_occupancy = et_occupancy * seasonal_occupancy

            # Step 12: Update seasonal feature using 3 × 5 moving average
            seasonal_co2 = modified_et_co2.rolling(window=3, center=True).mean().rolling(window=5, center=True).mean()
            seasonal_occupancy = modified_et_occupancy.rolling(window=3, center=True).mean().rolling(window=5, center=True).mean()

            # Step 13: Calculate reminder feature
            reminder_feature_co2 = preliminary_seasonal_adjusted_co2 / trend_co2
            reminder_feature_occupancy = preliminary_seasonal_adjusted_occupancy / trend_occupancy

    return trend_co2, trend_occupancy, seasonal_co2, seasonal_occupancy, reminder_feature_co2, reminder_feature_occupancy 